In [ ]:
import cmath
import math
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from sklearn import metrics

In [ ]:
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

def Convert_Mutual_Impedance(LA, LB, LC, LmAB, LmAC, LmBC):
    L1 = LA + LmAB + LmAC - LmBC
    L2 = LB + LmAB + LmBC - LmAC
    L3 = LC + LmAC + LmBC - LmAB
    return [L1, L2, L3]

def Convert_Mutual_Capacitace(CmAB, CmAC, CmBC, CA0, CB0, CC0):
    C1 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmBC) + CA0
    C2 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmAC) + CB0
    C3 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmAB) + CC0
    return [C1, C2, C3]


def Create_Complex(Ampli,Phase,inp='rad'):
    if (inp=='rad'):
        return complex(Ampli*math.cos(Phase),Ampli*math.sin(Phase))
    else:
        return complex(Ampli*math.cos(Phase*math.pi/180),Ampli*math.sin(Phase*math.pi/180))

def Complex_Characteristic_Impedance(R,L,G,C,w):
    return cmath.sqrt((complex(R,w*L))/(complex(G,w*C)))

def Complex_Propagation_Constant(R,L,G,C,w):
    return cmath.sqrt((complex(R,w*L))*(complex(G,w*C)))

In [ ]:
def compute_A(u1r, z1c, i1r):
    return (u1r - z1c * i1r)/2
def compute_B(u1r, z1c, i1r):
    return (u1r + z1c * i1r)/2

def compute_C(u1s, z1c, i1s, l, gama):
    return ((u1s - z1c * i1s)/2 ) * cmath.exp(gama *l)
def compute_D(u1s, z1c, i1s, l, gama):
    return ((u1s + z1c * i1s)/2 ) * cmath.exp(-gama *l)
def computeFaultLoc(l, a, b, c, d, gama):
    aux = cmath.log((a-c)/(d-b))/(2*gama*l)
    return l*(1- abs(aux))

In [ ]:
cmath.exp(complex(1,1))

In [ ]:
cmath.log(cmath.exp(1))

In [ ]:
data_normal = pd.read_excel('ComputeFaultLocation.xlsx',sheet_name='version_2_recompute')

In [ ]:
list(data_normal.columns)

In [ ]:
ListSim = list(data_normal['ID_Simulare'].unique())

In [ ]:
file_name = 'DirectComponent_val_Reactance_methode_Generator.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               '10','Err_10',
               '11','Err_11',
               '12','Err_12',
               '13','Err_13',
               '14','Err_14',
               '15','Err_15',
               '16','Err_16',
               '17','Err_17',
               '18','Err_18',
               '19','Err_19'
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            _Z_1_Compl = Create_Complex(float(_data['SD_Z1_val']),
                                       float(_data['SD_Z1_angle']),inp='rad')

            _gama_1_Compl = Create_Complex(float(_data['SD_Gama1_val']),
                                       float(_data['SD_Gama1_angle']),inp='rad')

            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_DirectComponent_Value']),
                                       float(_data['Vabc_generator_DirectComponent_Angle']),inp='rad')


            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_DirectComponent_Value']),
                                       float(_data['Vabc_consumator_DirectComponent_Angle']),inp='rad')


            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_DirectComponent_Value']),
                                       float(_data['Iabc_generator_DirectComponent_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_DirectComponent_Value']),
                                       float(_data['Iabc_consumator_DirectComponent_Angle']),inp='rad')
            
            Z_A_1 = _U_A_1_Compl / _I_A_1_Compl
            
            Z_A_2 = _U_A_2_Compl / _I_A_2_Compl
            
            d_A_1 = Z_A_1.imag / _Z_1_Compl.imag
            
            d_A_2 = Z_A_2.imag / _Z_1_Compl.imag          
            aux.append(abs(d_A_1))
            aux.append(abs(fault_value - abs(d_A_1))*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
file_name = 'DirectComponent_val_Reactance_methode_Consumator.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               '10','Err_10',
               '11','Err_11',
               '12','Err_12',
               '13','Err_13',
               '14','Err_14',
               '15','Err_15',
               '16','Err_16',
               '17','Err_17',
               '18','Err_18',
               '19','Err_19'
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            _Z_1_Compl = Create_Complex(float(_data['SD_Z1_val']),
                                       float(_data['SD_Z1_angle']),inp='rad')

            _gama_1_Compl = Create_Complex(float(_data['SD_Gama1_val']),
                                       float(_data['SD_Gama1_angle']),inp='rad')

            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_DirectComponent_Value']),
                                       float(_data['Vabc_generator_DirectComponent_Angle']),inp='rad')


            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_DirectComponent_Value']),
                                       float(_data['Vabc_consumator_DirectComponent_Angle']),inp='rad')


            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_DirectComponent_Value']),
                                       float(_data['Iabc_generator_DirectComponent_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_DirectComponent_Value']),
                                       float(_data['Iabc_consumator_DirectComponent_Angle']),inp='rad')
            
            Z_A_1 = _U_A_1_Compl / _I_A_1_Compl
            
            Z_A_2 = _U_A_2_Compl / _I_A_2_Compl
            
            d_A_1 = Z_A_1.imag / _Z_1_Compl.imag
            
            d_A_2 = Z_A_2.imag / _Z_1_Compl.imag              
            aux.append(abs(d_A_2))
            aux.append(abs(fault_value - abs(d_A_2))*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
file_name = 'DirectComponent_val_Chen_Methode.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               '10','Err_10',
               '11','Err_11',
               '12','Err_12',
               '13','Err_13',
               '14','Err_14',
               '15','Err_15',
               '16','Err_16',
               '17','Err_17',
               '18','Err_18',
               '19','Err_19'
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            _Z_1_Compl = Create_Complex(float(_data['SD_Z1_val']),
                                       float(_data['SD_Z1_angle']),inp='rad')

            _gama_1_Compl = Create_Complex(float(_data['SD_Gama1_val']),
                                       float(_data['SD_Gama1_angle']),inp='rad')

            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_DirectComponent_Value']),
                                       float(_data['Vabc_generator_DirectComponent_Angle']),inp='rad')


            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_DirectComponent_Value']),
                                       float(_data['Vabc_consumator_DirectComponent_Angle']),inp='rad')


            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_DirectComponent_Value']),
                                       float(_data['Iabc_generator_DirectComponent_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_DirectComponent_Value']),
                                       float(_data['Iabc_consumator_DirectComponent_Angle']),inp='rad')
            
            A_compute_phA = compute_A(_U_A_1_Compl, _Z_1_Compl, _I_A_1_Compl)
            B_compute_phA = compute_B(_U_A_1_Compl, _Z_1_Compl, _I_A_1_Compl)
            C_compute_phA = compute_C(_U_A_2_Compl, _Z_1_Compl, _I_A_2_Compl, Total_Length, _gama_1_Compl)
            D_compute_phA = compute_D(_U_A_2_Compl, _Z_1_Compl, _I_A_2_Compl, Total_Length, _gama_1_Compl)
            d_A_1 = computeFaultLoc(Total_Length, A_compute_phA, B_compute_phA, C_compute_phA, D_compute_phA, _gama_1_Compl)
                         
            
            aux.append(abs(d_A_1))
            aux.append(abs(fault_value - abs(d_A_1))*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))